In [8]:
import torch
import torchvision

class MyModel(torch.nn.Module):
  
   
  def __init__(self): 
     super().__init__()

     self.fc1= torch.nn.Linear(784,128)
     self.fc2 = torch.nn.Linear(128, 10)

  def forward(self ,x):
    #input shape 32*28*28
    x = x.reshape((x.shape[0],784))
    #output shape: 32*784
    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2 , train=True)
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x

In [9]:
# device =torch.device("cpu")
device = torch.device("cuda")
model = MyModel()
model = model.to(device)
model.train(True)


MyModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
# Hyper parameter
batch_size=64
epoch=10
lr=0.01

In [11]:

transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0),(1))
            # torchvision.transforms.RandomHorizontalFILIP(),              
])

dataset = torchvision.datasets.MNIST("./dataset",train =True,download=True, transform = transform )
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [15]:
#!rm -rf dataset # command for remove a folder in colab
#compile
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
#train
for epochs in range(epoch):
    for images , labels in train_data_loader:
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      # 1-forwarding
      preds = model(images) 
      # 2- backwarding
      loss = loss_function(preds,labels)
      loss.backward()
      # 3- update
      optimizer.step()

In [ ]:
#inference
model.train(False)